In [43]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 

In [2]:
attrs = ["amount", "freq", "avg_interval"]

In [3]:
raw_data = pd.read_csv("2022AIGO_H_LAB測試資料_雲端行動科技.csv")
raw_data.dropna(axis=0, subset=["item_tag"], inplace=True)

In [4]:
data_2020 = raw_data[raw_data["datetime"].str.startswith("2020")]
data_2020.loc[:, "datetime"] = pd.to_datetime(data_2020["datetime"], format="%Y-%m-%d")
data_2020["month"] = data_2020["datetime"].dt.month

c:\Users\kusod\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\kusod\AppData\Local\Temp/ipykernel_16420/605708320.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2020["month"] = data_2020["datetime"].dt.month


In [5]:
all_user = (data_2020.user_id.unique())

In [6]:
# df_2020 = data_2020[data_2020["item_tag"] != "優惠活動/折扣/集點"]
df_2020 = data_2020[data_2020["item_tag"] != "無法分類"]
df_2020 = df_2020[df_2020["item_tag"] != "餐飲需求"]

In [10]:
df_group_alltime = df_2020.groupby(["user_id", "item_tag"])[["amount", "datetime", "inv_num"]].agg(
    amount = ("amount", "sum"),
    freq = ("inv_num", lambda x : len(x.unique())),
    avg_interval = ("datetime", lambda x : x.sort_values().diff().sum().days / len(x))
)

In [97]:
df_ouput_alltime = pd.DataFrame(index = all_user, columns=[f'{i}_{j}'  for i in df_2020.item_tag.unique() for j in attrs])
def make_output_matrix(x):
    id = x.index.get_level_values("user_id")[0]
    x = x.droplevel((0))
    df_temp = x.stack()
    df_temp.index = [f"{x}_{y}" for x,y in df_temp.index]
    df_temp = df_temp.to_frame("").T
    df_ouput_alltime.loc[id, df_temp.columns] = df_temp.values

df_group_alltime.groupby(level = (0)).apply(make_output_matrix)

""


In [98]:
df_ouput_alltime.loc[:, df_ouput_alltime.columns.str.endswith("avg_interval")] = df_ouput_alltime.loc[:, df_ouput_alltime.columns.str.endswith("avg_interval")].replace([0, np.nan],1000)


In [99]:
df_ouput_alltime.fillna(0, inplace= True)

In [100]:
df_ouput_alltime = df_ouput_alltime.abs()

In [101]:
from sklearn.preprocessing import MinMaxScaler
minmax_sc = MinMaxScaler()
df_ouput_alltime_scaled = minmax_sc.fit_transform(df_ouput_alltime)

# No scaled

In [161]:

from sklearn.decomposition import NMF
K = 5
nmf = NMF(n_components= 5, init='random', random_state=0)
W = nmf.fit_transform(df_ouput_alltime)
H = nmf.components_

In [162]:
df_H = pd.DataFrame(H, columns=df_ouput_alltime.columns)
df_H

,乳製風味飲品_amount,乳製風味飲品_freq,乳製風味飲品_avg_interval,綠茶_amount,綠茶_freq,綠茶_avg_interval,停車費_amount,停車費_freq,停車費_avg_interval,碳酸飲料_amount,...,點鈔機_avg_interval,投影機_amount,投影機_freq,投影機_avg_interval,美甲玩具_amount,美甲玩具_freq,美甲玩具_avg_interval,美髮剪_amount,美髮剪_freq,美髮剪_avg_interval
0,6.157418,0.091954,0.000000,8.745778,0.132077,0.000000,2.060073,0.039767,0.000000,7.859318,...,2.427707,0.000000,0.00000,2.427707,0.004465,0.000017,2.427707,0.000000,0.000000,2.427707
1,8.005976,0.141236,22.904924,14.947583,0.379421,18.530382,4.020126,0.057528,33.224525,7.247672,...,39.068015,0.098645,0.00008,39.068015,0.009693,0.000036,39.068015,0.006223,0.000039,39.068015
2,1.001517,0.010676,0.665756,1.221876,0.016209,0.935488,0.433695,0.000000,1.076884,1.510404,...,1.587020,0.005114,0.00001,1.587020,0.005326,0.000020,1.587020,0.000309,0.000002,1.587020
3,1.008851,0.034805,0.000000,0.123252,0.003561,0.000000,0.308704,0.001596,1.695204,0.119820,...,1.644249,0.000000,0.00000,1.644249,0.002179,0.000008,1.644249,0.000000,0.000000,1.644249
4,0.706023,0.012771,0.472833,3.966017,0.070222,0.114055,1.426150,0.011046,1.003673,0.563600,...,2.240092,0.000000,0.00000,2.240092,0.000000,0.000000,2.240092,0.000000,0.000000,2.240092


In [173]:
for i in range(K):
    display(pd.DataFrame(pd.concat([df_H.loc[i, :].T.nlargest(5), df_H[df_H>0].loc[i, :].T.nsmallest(5)])))

,0
店家名稱_amount,846.934577
優惠活動/折扣/集點_amount,392.987338
其他服飾配件_amount,364.633630
其他料理_amount,133.996524
運動飲料_amount,85.724866
其他文具圖書_freq,0.000002
手環/手鍊_freq,0.000002
打卡鐘/紙_freq,0.000002
內接式硬碟_freq,0.000003
繃帶_freq,0.000004


,1
其他料理_amount,1.036831e+02
汽油_amount,8.845330e+01
智慧型手機_amount,6.165892e+01
香菸_amount,5.796673e+01
現沖咖啡_amount,5.492914e+01
電動沖牙機/噴頭_freq,9.212893e-07
燕窩_freq,5.236330e-06
洗手機_freq,8.253232e-06
電視天線_freq,1.625989e-05
其他茶具用品_freq,1.860698e-05


,2
襪子_amount,3.355875e+02
口腔特殊護理/清潔_amount,2.172663e+02
卸妝油/乳/水_amount,2.163928e+02
優惠活動/折扣/集點_amount,1.827069e+02
妝前乳/隔離霜_amount,1.752554e+02
點鈔機_freq,9.358993e-09
電視天線_freq,2.109515e-08
顯示器保護鏡_freq,2.984005e-07
床架/板/框/底_freq,3.124986e-07
勺_freq,3.964118e-07


,3
手續/服務/外送費_amount,5.826576e+02
運費_amount,4.531463e+02
筆記型電腦_amount,4.535203e+01
優惠活動/折扣/集點_amount,1.046623e+01
乳清蛋白_amount,5.862233e+00
電動洗臉機_freq,1.427621e-07
電視_freq,1.605096e-07
馬桶設備_freq,1.743064e-07
桌球_freq,2.055091e-07
燈飾_freq,2.740678e-07


,4
電費_amount,6.705636e+02
其他料理_amount,9.598258e+01
汽油_amount,1.250752e+01
現沖咖啡_amount,1.110576e+01
儲值現金/點數_amount,8.842493e+00
氣炸鍋_freq,4.500023e-07
其他文具圖書_freq,6.286909e-07
紹興/紅露/花雕酒_freq,8.380250e-07
藥盒_freq,9.826575e-07
洗臉周邊_freq,1.357813e-06


In [172]:
for i in range(K):
    display(pd.DataFrame(pd.concat([df_H.loc[i, df_H.columns.str.endswith("amount")].T.nlargest(5), df_H[df_H>0].loc[i, df_H.columns.str.endswith("amount")].T.nsmallest(5)])))
    display(pd.DataFrame(pd.concat([df_H.loc[i, df_H.columns.str.endswith("freq")].T.nlargest(5), df_H[df_H>0].loc[i, df_H.columns.str.endswith("freq")].T.nsmallest(5)])))
    display(pd.DataFrame(pd.concat([df_H.loc[i, df_H.columns.str.endswith("interval")].T.nlargest(5), df_H[df_H>0].loc[i, df_H.columns.str.endswith("interval")].T.nsmallest(5)])))

,0
店家名稱_amount,846.934577
優惠活動/折扣/集點_amount,392.987338
其他服飾配件_amount,364.633630
其他料理_amount,133.996524
運動飲料_amount,85.724866
其他文具圖書_amount,0.000044
履歷表_amount,0.000225
繃帶_amount,0.000304
奶精_amount,0.000325
汽車電力系統_amount,0.000342


,0
優惠活動/折扣/集點_freq,0.993033
其他料理_freq,0.392523
麵包/漢堡/蛋糕_freq,0.367479
手續/服務/外送費_freq,0.343817
店家名稱_freq,0.321438
其他文具圖書_freq,0.000002
手環/手鍊_freq,0.000002
打卡鐘/紙_freq,0.000002
內接式硬碟_freq,0.000003
繃帶_freq,0.000004


,0
黑/白麥汁_avg_interval,2.589501
眼唇卸妝_avg_interval,2.528820
孕婦保養_avg_interval,2.514274
蜆精華_avg_interval,2.481839
其他頭髮保養/清潔_avg_interval,2.476001
海苔_avg_interval,0.017240
鱈魚絲/海鮮酥_avg_interval,0.038238
仙草/愛玉/豆花_avg_interval,0.041316
泡菜_avg_interval,0.073755
襪子_avg_interval,0.089258


,1
其他料理_amount,103.683125
汽油_amount,88.453299
智慧型手機_amount,61.658924
香菸_amount,57.966728
現沖咖啡_amount,54.929138
其他茶具用品_amount,0.000540
奶精_amount,0.000551
防污/防潮貼/墊_amount,0.000792
其他文具圖書_amount,0.000834
其他袋子_amount,0.000894


,1
優惠活動/折扣/集點_freq,1.848883e+00
麵包/漢堡/蛋糕_freq,7.646063e-01
其他料理_freq,5.433020e-01
現沖咖啡_freq,5.330911e-01
奶茶_freq,4.403185e-01
電動沖牙機/噴頭_freq,9.212893e-07
燕窩_freq,5.236330e-06
洗手機_freq,8.253232e-06
電視天線_freq,1.625989e-05
其他茶具用品_freq,1.860698e-05


...

,3
優惠活動/折扣/集點_freq,7.484097e-01
運費_freq,7.091774e-01
手續/服務/外送費_freq,6.158185e-01
麵包/漢堡/蛋糕_freq,1.301938e-01
瓶/罐裝多多_freq,4.768306e-02
電動洗臉機_freq,1.427621e-07
電視_freq,1.605096e-07
馬桶設備_freq,1.743064e-07
桌球_freq,2.055091e-07
燈飾_freq,2.740678e-07


,3
生鮮蔬菜_avg_interval,1.780418
衛生棉_avg_interval,1.745538
鮮蛋_avg_interval,1.742923
口香糖/薄荷糖_avg_interval,1.738081
調酒/果實酒_avg_interval,1.734972
瓶/罐裝多多_avg_interval,0.154844
雞料理_avg_interval,0.170135
其他麵食_avg_interval,0.197361
義大利麵_avg_interval,0.200410
壽司/飯糰_avg_interval,0.218255


,4
電費_amount,670.563554
其他料理_amount,95.982585
汽油_amount,12.507519
現沖咖啡_amount,11.105758
儲值現金/點數_amount,8.842493
其他文具圖書_amount,0.000013
藥盒_amount,0.000038
奶精_amount,0.000083
繃帶_amount,0.000372
廚房清潔/疏通劑_amount,0.000379


,4
其他料理_freq,2.443302e-01
優惠活動/折扣/集點_freq,2.425771e-01
手續/服務/外送費_freq,2.086719e-01
現沖咖啡_freq,1.471635e-01
電費_freq,8.300536e-02
氣炸鍋_freq,4.500023e-07
其他文具圖書_freq,6.286909e-07
紹興/紅露/花雕酒_freq,8.380250e-07
藥盒_freq,9.826575e-07
洗臉周邊_freq,1.357813e-06


,4
能量飲料_avg_interval,2.353921
鐵觀音_avg_interval,2.328860
外套_avg_interval,2.323786
麥茶_avg_interval,2.322376
兒童紙尿褲_avg_interval,2.306150
碳酸飲料_avg_interval,0.056680
現沖咖啡_avg_interval,0.066437
綠茶_avg_interval,0.114055
其他零食_avg_interval,0.130430
鍋物/湯品_avg_interval,0.156394


# Scaled

In [112]:
K = 5
nmf_sc = NMF(n_components= K, init='random', random_state=0, max_iter = 1000)
W_sc = nmf_sc.fit_transform(df_ouput_alltime_scaled)
H_sc = nmf_sc.components_

c:\Users\kusod\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 1000 reached. Increase it to improve convergence.
  warnings.warn(


In [113]:
df_H_sc = pd.DataFrame(H_sc, columns=df_ouput_alltime.columns)
df_H_sc

,乳製風味飲品_amount,乳製風味飲品_freq,乳製風味飲品_avg_interval,綠茶_amount,綠茶_freq,綠茶_avg_interval,停車費_amount,停車費_freq,停車費_avg_interval,碳酸飲料_amount,...,點鈔機_avg_interval,投影機_amount,投影機_freq,投影機_avg_interval,美甲玩具_amount,美甲玩具_freq,美甲玩具_avg_interval,美髮剪_amount,美髮剪_freq,美髮剪_avg_interval
0,0.082832,0.073831,0.000000,0.104585,0.131383,0.000000,0.011101,0.008082,1.077812,0.079717,...,0.0,0.003642,0.000531,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,0.000000,0.000000,0.800369,0.000000,0.000000,0.211830,0.000000,0.001183,0.312196,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.634523,0.000000,...,0.0,0.000592,0.006306,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
3,0.148553,0.092698,0.064172,0.117557,0.080949,0.064916,0.026854,0.028991,0.286568,0.120668,...,0.0,0.000471,0.001487,0.0,0.005654,0.005654,0.0,0.004496,0.004496,0.0
4,0.000000,0.000000,1.269664,0.000000,0.000000,1.895619,0.000000,0.000000,0.704448,0.000000,...,0.0,0.000000,0.000000,0.0,0.000585,0.000585,0.0,0.001920,0.001920,0.0


In [176]:
for i in range(K):
    display(pd.DataFrame(pd.concat([df_H_sc.loc[i, :].T.nlargest(5), df_H_sc[df_H_sc>0].loc[i, :].T.nsmallest(5)])))

,0
T恤_avg_interval,1.897863e+00
化妝水/噴霧_avg_interval,1.881936e+00
襪子_avg_interval,1.875019e+00
精華液/油/乳/霜_avg_interval,1.874021e+00
長褲_avg_interval,1.850892e+00
平板保護周邊_amount,5.461350e-07
固態硬碟_freq,4.992316e-05
仙草/愛玉/豆花_amount,5.307647e-05
乳清蛋白_amount,7.539137e-05
毛衣_freq,1.078065e-04


,1
巧克力_avg_interval,1.516195
糖果_avg_interval,1.446999
蛋捲/夾心餅/蛋黃派_avg_interval,1.375996
奶酪/布丁/果凍_avg_interval,1.307964
瓶/罐裝多多_avg_interval,1.279701
兒童口腔清潔_amount,0.000006
氣泡水機周邊_freq,0.000008
其他傢俱_amount,0.000073
床架/板/框/底_amount,0.000108
床架/板/框/底_freq,0.000108


,2
生豬肉_avg_interval,2.143882
生鮮海產_avg_interval,1.902605
生鮮蔬菜_avg_interval,1.856933
豆皮/豆腐_avg_interval,1.849470
生雞肉_avg_interval,1.781371
其他保健食品_freq,0.000004
抹布_freq,0.000056
便利貼/索引貼_freq,0.000056
電視_amount,0.000076
製冰機_freq,0.000227


,3
其他酒_avg_interval,0.725278
香菸_avg_interval,0.721309
打火機_avg_interval,0.716116
蜆精華_avg_interval,0.712831
琴酒_avg_interval,0.710980
琴酒_amount,0.000005
CPU散熱周邊_freq,0.000038
簡報筆_amount,0.000059
簡報筆_freq,0.000153
交換器_amount,0.000315


,4
奶茶_avg_interval,2.039307
壽司/飯糰_avg_interval,2.031570
雞料理_avg_interval,2.001107
紅茶_avg_interval,1.960302
其他飲料_avg_interval,1.929898
整線器/綁物帶_amount,0.000019
去漬/殺菌劑_freq,0.000061
數位相機_freq,0.000064
火腿/培根_freq,0.000065
保鮮膜/蓋_freq,0.000081


In [178]:
for i in range(K):
    display(pd.DataFrame(pd.concat([df_H_sc.loc[i, df_H_sc.columns.str.endswith("amount")].T.nlargest(5), df_H_sc[df_H_sc>0].loc[i, df_H_sc.columns.str.endswith("amount")].T.nsmallest(5)])))
    display(pd.DataFrame(pd.concat([df_H_sc.loc[i, df_H_sc.columns.str.endswith("freq")].T.nlargest(5), df_H_sc[df_H_sc>0].loc[i, df_H_sc.columns.str.endswith("freq")].T.nsmallest(5)])))
    display(pd.DataFrame(pd.concat([df_H_sc.loc[i, df_H_sc.columns.str.endswith("interval")].T.nlargest(5), df_H_sc[df_H_sc>0].loc[i, df_H_sc.columns.str.endswith("interval")].T.nsmallest(5)])))

,0
紅茶_amount,1.175713e-01
雞料理_amount,1.118401e-01
壽司/飯糰_amount,1.071467e-01
綠茶_amount,1.045851e-01
其他飯食_amount,9.423845e-02
平板保護周邊_amount,5.461350e-07
仙草/愛玉/豆花_amount,5.307647e-05
乳清蛋白_amount,7.539137e-05
其他舒壓好眠_amount,1.294510e-04
底片_amount,1.785507e-04


,0
優惠活動/折扣/集點_freq,0.174074
麵包/漢堡/蛋糕_freq,0.162556
綠茶_freq,0.131383
熱狗/香腸_freq,0.131172
其他飯食_freq,0.111253
固態硬碟_freq,0.000050
毛衣_freq,0.000108
底片_freq,0.000179
電烤盤_freq,0.000198
網球_freq,0.000206


,0
T恤_avg_interval,1.897863
化妝水/噴霧_avg_interval,1.881936
襪子_avg_interval,1.875019
精華液/油/乳/霜_avg_interval,1.874021
長褲_avg_interval,1.850892
生鮮水果_avg_interval,0.002800
茶葉蛋_avg_interval,0.006768
運動飲料_avg_interval,0.016483
瓶/罐裝咖啡_avg_interval,0.020917
巧克力_avg_interval,0.025574


,1
其他料理_amount,0.017103
短褲_amount,0.015508
平板電腦_amount,0.014952
襯衫_amount,0.013414
游泳_amount,0.011687
兒童口腔清潔_amount,0.000006
其他傢俱_amount,0.000073
床架/板/框/底_amount,0.000108
淨水器_amount,0.000116
感應燈/磁吸燈_amount,0.000130


,1
雲端空間_freq,0.030561
手套_freq,0.028645
吹風機_freq,0.025747
睡衣/居家服_freq,0.021311
T恤_freq,0.018809
氣泡水機周邊_freq,0.000008
床架/板/框/底_freq,0.000108
微單眼/類單眼/單眼相機_freq,0.000124
杯墊/餐墊/隔熱墊_freq,0.000190
盤_freq,0.000234


...

,3
牙膏/牙粉_freq,0.309834
優惠活動/折扣/集點_freq,0.262080
麵包/漢堡/蛋糕_freq,0.258564
牙刷_freq,0.231691
膠帶/黏貼用品_freq,0.223912
CPU散熱周邊_freq,0.000038
簡報筆_freq,0.000153
桌機供電設備_freq,0.000521
打卡鐘/紙_freq,0.000643
點鈔機_freq,0.000738


,3
其他酒_avg_interval,0.725278
香菸_avg_interval,0.721309
打火機_avg_interval,0.716116
蜆精華_avg_interval,0.712831
琴酒_avg_interval,0.710980
洗髮清潔_avg_interval,0.003204
奶茶_avg_interval,0.003374
火鍋料_avg_interval,0.010720
鮮蛋_avg_interval,0.017126
濕巾_avg_interval,0.019243


,4
沾拌醬_amount,0.021269
紫菜/昆布_amount,0.018031
鮮蛋_amount,0.017252
鹽/辛香料_amount,0.017233
生麵條_amount,0.016020
整線器/綁物帶_amount,0.000019
掃地機器人周邊_amount,0.000096
冰熱敷墊/袋_amount,0.000178
醬瓜類罐頭_amount,0.000221
其他咖啡用具_amount,0.000326


,4
植物油_freq,0.035597
豆皮/豆腐_freq,0.024667
紫菜/昆布_freq,0.024050
糖/果糖_freq,0.022683
烘焙粉_freq,0.018879
去漬/殺菌劑_freq,0.000061
數位相機_freq,0.000064
火腿/培根_freq,0.000065
保鮮膜/蓋_freq,0.000081
紙巾_freq,0.000214


,4
奶茶_avg_interval,2.039307
壽司/飯糰_avg_interval,2.031570
雞料理_avg_interval,2.001107
紅茶_avg_interval,1.960302
其他飲料_avg_interval,1.929898
火鍋料_avg_interval,0.006222
生豬肉_avg_interval,0.011580
米_avg_interval,0.020678
生鮮海產_avg_interval,0.047598
生麵條_avg_interval,0.051842


In [179]:
K = 10
nmf_sc = NMF(n_components= 10, init='random', random_state=0, max_iter = 1000)
W_sc = nmf_sc.fit_transform(df_ouput_alltime_scaled)
H_sc = nmf_sc.components_

c:\Users\kusod\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 1000 reached. Increase it to improve convergence.
  warnings.warn(


In [181]:
df_H_sc = pd.DataFrame(H_sc, columns=df_ouput_alltime.columns)
df_H_sc

,乳製風味飲品_amount,乳製風味飲品_freq,乳製風味飲品_avg_interval,綠茶_amount,綠茶_freq,綠茶_avg_interval,停車費_amount,停車費_freq,停車費_avg_interval,碳酸飲料_amount,...,點鈔機_avg_interval,投影機_amount,投影機_freq,投影機_avg_interval,美甲玩具_amount,美甲玩具_freq,美甲玩具_avg_interval,美髮剪_amount,美髮剪_freq,美髮剪_avg_interval
0,0.000000,0.000000,0.230504,0.000000,0.000000,0.000000,0.003795,0.000000,0.652479,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,0.000000,0.000000,0.789014,0.000000,0.000000,0.000000,0.000000,0.003173,0.289084,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.611867,0.000000,...,0.0,0.003411,0.005854,0.0,0.001266,0.001266,0.0,0.004726,0.004726,0.0
3,0.101763,0.085425,0.000000,0.091154,0.113955,0.000000,0.009080,0.009450,0.271933,0.081837,...,0.0,0.003118,0.002785,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
4,0.000000,0.000000,2.374164,0.000000,0.000000,1.314808,0.000000,0.000000,0.341561,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
5,0.000000,0.000000,0.000000,0.020905,0.016963,0.000000,0.000000,0.000000,0.229865,0.053012,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
6,0.018889,0.010832,0.042915,0.025086,0.004735,0.000000,0.011369,0.010960,0.276962,0.000000,...,0.0,0.000000,0.000000,0.0,0.002647,0.002647,0.0,0.003260,0.003260,0.0
7,0.108036,0.061611,0.074591,0.083630,0.057841,0.000000,0.015546,0.017071,0.000000,0.090155,...,0.0,0.000000,0.000000,0.0,0.003958,0.003958,0.0,0.000000,0.000000,0.0
8,0.000000,0.000000,0.329665,0.000000,0.000000,1.181443,0.000847,0.001322,0.208019,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
9,0.000892,0.003647,0.000000,0.018372,0.025565,0.000000,0.000000,0.000000,0.397311,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0


In [182]:
for i in range(K):
    display(pd.DataFrame(pd.concat([df_H_sc.loc[i, :].T.nlargest(5), df_H_sc[df_H_sc>0].loc[i, :].T.nsmallest(5)])))

,0
義大利麵_avg_interval,1.780005
餃子/鍋貼/蒸餃_avg_interval,1.594390
焗飯_avg_interval,1.555621
粥_avg_interval,1.553423
燴飯_avg_interval,1.503681
地毯/止滑墊/腳踏墊_amount,0.000002
其他生鮮_amount,0.000037
防污/防潮貼/墊_amount,0.000038
課程/考試費_freq,0.000039
眼鏡周邊_freq,0.000060


,1
蛋捲/夾心餅/蛋黃派_avg_interval,2.127056
巧克力_avg_interval,1.777874
糖果_avg_interval,1.545621
洋芋類零食_avg_interval,1.527506
蘇打餅/煎餅/方塊酥_avg_interval,1.470393
其他視聽電子_freq,0.000003
其他廚房用品_amount,0.000059
潤滑液_amount,0.000077
鳳梨酥_amount,0.000092
潽洱茶_freq,0.000096


,2
手續/服務/外送費_avg_interval,1.623738
店家名稱_avg_interval,1.480804
薯餅/薯條/可樂餅_avg_interval,1.378217
鍋物/湯品_avg_interval,1.252864
雞料理_avg_interval,0.769751
洗髮清潔_amount,0.000010
啤酒_freq,0.000050
真空保鮮機_freq,0.000093
其他活力元氣_amount,0.000101
手機保護周邊_freq,0.000104


,3
精華液/油/乳/霜_avg_interval,0.543707
化妝水/噴霧_avg_interval,0.541516
T恤_avg_interval,0.537718
童裝_avg_interval,0.533321
裙類_avg_interval,0.526167
刀具砧板_amount,0.000008
其他婦嬰用品_amount,0.000011
電腦軟體_amount,0.000030
除毛球機_freq,0.000044
其他保健食品_freq,0.000055


,4
壽司/飯糰_avg_interval,5.932067
堅果/穀物飲_avg_interval,5.904770
茶葉蛋_avg_interval,5.289276
現沖咖啡_avg_interval,4.728997
其他料理_avg_interval,4.581595
皮蛋_amount,0.000025
無線耳機_amount,0.000032
便利貼/索引貼_freq,0.000045
生羊肉_amount,0.000163
衣架/衣夾_amount,0.000306


,5
礦泉水_avg_interval,1.700227
菜料理_avg_interval,1.671302
鹼性水_avg_interval,1.665487
純水_avg_interval,1.567504
沙拉_avg_interval,1.438672
手搖多多飲品_freq,0.000075
水桶/水杓_amount,0.000076
薯餅/薯條/可樂餅_amount,0.000083
家飾/擺飾_amount,0.000139
監視器_amount,0.000165


,6
炒麵_avg_interval,0.424639
燈泡/燈管/崁燈_avg_interval,0.424578
焗飯_avg_interval,0.421563
橡皮擦_avg_interval,0.412523
麵筋類罐頭_avg_interval,0.409757
蘋果派_freq,0.000006
貓用品_freq,0.000009
乳清蛋白_amount,0.000012
蓮蓬頭配件_amount,0.000017
睡袋_freq,0.000021


,7
面膜/凍膜/泥膜_avg_interval,0.411806
底妝_avg_interval,0.389048
隱形眼鏡_avg_interval,0.386572
其他美妝美體_avg_interval,0.386241
卸妝油/乳/水_avg_interval,0.383286
杯套/袋_amount,0.000014
曬衣桿/網/繩_amount,0.000069
玻璃清潔/保養劑_freq,0.000269
工具鉗_amount,0.000301
吸塵器周邊_freq,0.000318


,8
烏龍茶_avg_interval,1.405058
紅茶_avg_interval,1.206535
綠茶_avg_interval,1.181443
青茶_avg_interval,1.133817
運動飲料_avg_interval,1.080156
衣領/袖專用洗劑_freq,0.000016
卸甲液/棉_amount,0.000033
其他豆製品/素料_freq,0.000048
底片_amount,0.000067
底片_freq,0.000067


,9
生豬肉_avg_interval,3.833029
豆皮/豆腐_avg_interval,3.337927
生鮮海產_avg_interval,3.210404
鮮蛋_avg_interval,2.974262
生雞肉_avg_interval,2.968863
漫畫_amount,0.000039
漫畫_freq,0.000039
VR頭戴裝置_amount,0.000039
VR頭戴裝置_freq,0.000039
修正帶/液_amount,0.000056
